In [3]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

C:\Users\91706\AppData\Roaming\Python\Python37\site-packages\paramiko\transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


Ready to use Azure ML 1.43.0 to work with akashbws


In [2]:
from azureml.core import Workspace, Dataset
dataset = Dataset.get_by_name(ws, name='salary_classification')
df = dataset.to_pandas_dataframe()

## Creating Environment

In [13]:
from azureml.core import Experiment, ScriptRunConfig, Environment
env = Environment.from_conda_specification("adult_classification_env", "D:/Adult Classification Project/Adult-Income-Classification/environment_droplet.yml")

In [14]:
## if using local
from azureml.core import Environment

myenv = Environment("user-managed-env")
myenv.python.user_managed_dependencies = True

In [9]:
%%writefile dataprep_experiment.py
import azureml.core
from azureml.core import Workspace, Dataset, Run
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib_inline


# Load the workspace from the saved config file
ws = Workspace.from_config()
print(ws)



# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
dataset = Dataset.get_by_name(ws, name='salary_classification')
df = dataset.to_pandas_dataframe()


# Count the rows and log the result
row_count = (len(df))
run.log('observations', row_count)
print('Analyzing {} rows of data'.format(row_count))

# Count and log the label counts
salary_counts = df['salary'].value_counts()
print(salary_counts)
for k, v in salary_counts.items():
    run.log('Label:' + str(k), v)

## imbalanced dataset figure
fig = plt.figure(figsize=(6,6))
sns.countplot(x = 'salary',data = df)
plt.show()
run.log_image(name='label distribution', plot=fig)

#percentage of imbalanceness
print(f"<= 50k : {round(24720 /32561 * 100 , 2)}")
print(f"> 50k : {round(7841 /32561 * 100 , 2)}")

run.log('% of <= 50k',round(24720 /32561 * 100 , 2))
run.log('% of > 50k', round(7841 /32561 * 100 , 2))


# Log summary statistics for numeric columns
med_columns = ['age','fnlwgt', 'education-num','capital-gain', 'capital-loss', 'hours-per-week']
summary_stats = df[med_columns].describe().to_dict()
for col in summary_stats:
    keys = list(summary_stats[col].keys())
    values = list(summary_stats[col].values())
    for index in range(len(keys)):
        run.log_row(col, stat=keys[index], value = values[index])
        
# Log summary statistics for cat columns
cat_columns = ['workclass','education','marital-status', 'occupation', 'relationship','race', 'sex','country','salary']
cat_summary_stats = df[cat_columns].describe().to_dict()
for col in cat_summary_stats:
    keys = list(cat_summary_stats[col].keys())
    values = list(cat_summary_stats[col].values())
    for index in range(len(keys)):
        run.log_row(col, stat=keys[index], value = values[index])
        
# Complete the run
run.complete()

Overwriting dataprep_experiment.py


In [7]:
import azureml.core
from azureml.core import Experiment, ScriptRunConfig, Environment,Workspace
from azureml.core.runconfig import DockerConfiguration
from azureml.widgets import RunDetails
import os, shutil


# Load the workspace from the saved config file
ws = Workspace.from_config()

# Create a folder for the experiment files
folder_name = 'research_env'
experiment_folder = './'

# Create a Python environment for the experiment (from a .yml file)
env = Environment.from_conda_specification("adult_income_environment", "D:/Adult Classification Project/Adult-Income-Classification/environment.yml")

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='dataprep_experiment.py',
                                environment = env)
                                

# submit the experiment
experiment = Experiment(workspace=ws, name='adult_income_classification')
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'adult_income_classification_1658546751_9c75769c',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2022-07-23T03:26:01.156054Z',
 'services': {},
 'error': {'error': {'code': 'UserError',
   'message': "User program failed with FileNotFoundError: [Errno 2] No such file or directory: 'C:\\\\Users\\\\91706\\\\AppData\\\\Local\\\\Temp\\\\azureml_runs\\\\adult_income_classification_1658546751_9c75769c\\\\label_distribution.png'",
   'messageParameters': {},
   'detailsUri': 'https://aka.ms/azureml-run-troubleshooting',
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '36861fc1-ceea-4cb1-97f0-a2086a84c1df',
  'azureml.git.repository_uri': 'https://github.com/AkashBorgalli/Adult-Income-Classification.git',
  'mlflow.source.git.repoURL': 'https://github.com/AkashBorgalli/Adult-Income-Classification.git',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azurem

In [10]:
from azureml.core import Workspace
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

ws = Workspace.from_config() # This automatically looks for a directory .azureml

# Choose a name for your CPU cluster
cpu_cluster_name = "classification-cpu-cluster"

# Verify that the cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4, 
                                                           idle_seconds_before_scaledown=2400)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## registering env

In [11]:
from azureml.core import Experiment, ScriptRunConfig, Environment
experiment_env = Environment.from_conda_specification("adult_classification_env", "D:/Adult Classification Project/Adult-Income-Classification/environment.yml")
experiment_env.register(ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20220616.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "adult_classification_env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "dependencies": [
                "python=3.6.2",
           

## creating pipeline

In [13]:
%%writefile prep_adult_income.py
# Import libraries
import os
import argparse
import pandas as pd
import seaborn as sns
import numpy as np
import azureml.core
from azureml.core import Workspace, Dataset, Run
from imblearn.combine import SMOTEENN
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib_inline


# Get parameters
parser = argparse.ArgumentParser()
parser.add_argument("--input-data", type=str, dest='raw_dataset_id', help='raw dataset')
parser.add_argument('--prepped-data', type=str, dest='prepped_data', default='prepped_data', help='Folder for results')
args = parser.parse_args()
save_folder = args.prepped_data
print('created folder name for storing prepped data as', save_folder)

# Load the workspace from the saved config file
ws = Workspace.from_config()

# Get the experiment run context
run = Run.get_context()

# load the data (passed as an input dataset)
print("Loading Data...")
df = run.input_datasets['raw_data'].to_pandas_dataframe()

#wks = run.experiment.workspace
#raw_dataset = Dataset.get_by_id(wks, id=args.raw_dataset_id)
#df = raw_dataset.to_pandas_dataframe()

##dataset = Dataset.get_by_name(ws, name='salary_classification')
##df = dataset.to_pandas_dataframe()


# Log raw row count
row_count = (len(df))
run.log('raw_rows', row_count)
print("Logging row count:",row_count)

# we will apply log transformation on age and fnlwgt but not oncapital loss, capital gain as log(0) will give n.d
print('Applying Log Transformation to age and fnlwgt columns')
df["age"] = np.log(df["age"])
df["fnlwgt"] = np.log(df["fnlwgt"])


# missing percentage values for following columns:
print('missing percentage values for following columns are:')
print(f"workclass : {round(2093 / 32561 , 4) *100}%")
print(f"occupation : {round(1843 / 32561 , 4) *100}%")
print(f"native-country : {round(583 / 32561 , 4) *100}%")

## filling with modes
print('filling missing values by mode on workclass, occupation and country columns')
df["workclass"] = df['workclass'].str.replace('?', 'Private' )
df['occupation'] = df['occupation'].str.replace('?', 'Prof-specialty' )
df['country'] = df['country'].str.replace('?', 'United-States')

# reduced education unique categories
print('Reducing the unique counts from educationan and marital Status')
df["education"].replace(['Preschool', '1st-4th', '5th-6th', '7th-8th', '9th','10th', '11th', '12th'], 'school' ,
                         inplace = True , regex = True)
df["education"].replace(['Assoc-voc', 'Assoc-acdm', 'Prof-school', 'Some-college'], 'higher' , inplace = True , regex = True)

#reduced mariatal-status unique categories
df['marital-status'].replace(['Married-civ-spouse', 'Married-AF-spouse'], 'married' , inplace = True , regex = True)
df['marital-status'].replace(['Divorced', 'Separated','Widowed',
                                                   'Married-spouse-absent'], 'other' , inplace = True , regex = True)
# converting salary columns to binary
print('Converting salary feature to binary')
df["salary"] = df["salary"].replace({'<=50K' : 0 , ">50K" : 1 } , regex = True)

#checking the unique count now
cat_columns = df.select_dtypes(include='object')
cat_columns.columns
## checking the reduced unique count now
print('Checking the reduced unique count now')
for feature in cat_columns.columns:
    print(f" {feature}  :  {len(df[feature].unique())}")
    

# plotting heatmap
print('plotting heat map')
heatmap_fig = plt.figure(figsize=(20,10), dpi = 150)
sns.heatmap(df.corr(), annot = True, cmap = 'viridis')
plt.show()
run.log_image(name='correlation plot', plot=heatmap_fig)



## applied label encoding for entire dataset
print('applying label encoding for entire dataset')
from sklearn.preprocessing import  LabelEncoder
df = df.apply(LabelEncoder().fit_transform)


## splitting the data
print('Splitting the data in X and y')
X = df.drop(['salary'], axis =1)
y = df['salary']

# handling imbalanced dataset
print('Handling imbalanced dataset')
SMOTEENN = SMOTEENN(n_jobs=-1)
print('Original dataset shape %s' % Counter(y))
X_res, y_res = SMOTEENN.fit_resample(X, y)
print('After undersample dataset shape %s' % Counter(y_res))
labeldf = pd.DataFrame(y_res,columns=['salary'])

##label count after undersampling
label_counts = Counter(y_res)
print('label count of <=50k ', label_counts[0])
print('label count of >50k ', label_counts[1])

# plotting balanced data graph
print('plotting balanced data graph')
balanced_data_fig = plt.figure(figsize=(6,6))
sns.countplot(labeldf['salary'])
##plt.show()
run.log_image(name='Balanced salary distribution', plot=balanced_data_fig)

#clubbing the data
print('clubbing the data to be used for training')
processed_df = pd.concat([X_res, y_res], axis=1)

# Log processed rows
processed_row_count = len(processed_df)
run.log('processed_rows', processed_row_count)
print('Transformed {} rows of data'.format(processed_row_count))

#percentage of balanceness
print('percentages of balanced data..')
print('---------------------------------------------------------------------------------------------------------------------')
print(f"<= 50k : {round(label_counts[0] /processed_row_count * 100 , 2)}")
print(f"> 50k : {round(label_counts[1] /processed_row_count * 100 , 2)}")

run.log('% of <= 50k',round(label_counts[0] /processed_row_count * 100 , 2))
run.log('% of > 50k', round(label_counts[1] /processed_row_count * 100 , 2))


# Save the prepped data
print("Saving Data...")
os.makedirs(save_folder, exist_ok=True)
save_path = os.path.join(save_folder,'data.csv')
df.to_csv(save_path, index=False, header=True)

# End the run
run.complete()

Overwriting prep_adult_income.py


In [14]:
%%writefile train_adult_salary.py
# Import libraries
from azureml.core import Run, Model
import argparse
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import roc_auc_score,f1_score,accuracy_score,precision_score,recall_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, plot_confusion_matrix,ConfusionMatrixDisplay
from sklearn.metrics import precision_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import matplotlib_inline
import seaborn as sns
from azureml.interpret import ExplanationClient
from interpret.ext.blackbox import TabularExplainer

labels = ['<=50K', '>50K']
features = ['age', 'workclass', 'fnlwgt', 'education', 'education-num','marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'country']
# Get parameters
parser = argparse.ArgumentParser()
parser.add_argument("--training-data", type=str, dest='training_data', help='training data')
args = parser.parse_args()
training_data = args.training_data
print('training data', training_data)



# Get the experiment run context
run = Run.get_context()

# load the prepared data file in the training folder
print("Loading Data for Training...")
file_path = os.path.join(training_data,'data.csv')
df = pd.read_csv(file_path)

# Separate features and labels
print("Splitting data X and Y...")
X = df.drop(['salary'], axis =1)
y = df['salary']

# Split data into training set and test set
print("Splitting data into X_train and y_train...")
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 101)

# scaling the data
print("Performing Standard Scalar...")
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Training a lightGBM model
print('Training a LightGBM Classifier model...')
clf = lgb.LGBMClassifier(boosting_type='goss',objective='binary',n_jobs=-1,n_estimators=200)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print("The ROC-AUC Score obtained on is : " , roc_auc_score(y_test, y_pred))
print("The Macro F1-Score obtained on is : " , f1_score(y_test,  y_pred,average = 'macro'))
print("The F1 scores of each class on are : ",f1_score(y_test,  y_pred,average = None))

# calculate accuracy
acc = np.average(y_pred == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = clf.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))


# calculate precision
precisionscore = precision_score(y_test, y_pred)
print('Precision: ' + str(precisionscore))
run.log('Precision', precisionscore)

# plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

#plot confusion matrix
fig = plt.figure(figsize=(6,6))
cm = confusion_matrix(y_test, y_pred )
plt.title('Heatmap of Confusion Matrix', fontsize = 12)
sns.heatmap(cm, annot = True ,  fmt = "d")
##run.log_confusion_matrix(name = "Heatmap of Confusion Matrix", value = cm)
run.log_image(name='Confusion Matrix LGBM', plot=fig)


# Save the trained model in the outputs folder
print("Saving model...")
os.makedirs('outputs', exist_ok=True)
model_file = os.path.join('outputs', 'adult_income_model.pkl')
joblib.dump(value=clf, filename=model_file)
# Register the model
print('Registering model...')
Model.register(workspace=run.experiment.workspace,
               model_path = model_file,
               model_name = 'adult_income_model',
               tags={'Training context':'Pipeline'},
               properties={'AUC': np.float(auc), 'Accuracy': np.float(acc), 'Precision': precision_score(y_test, y_pred)})

# Get explanation
explainer = TabularExplainer(clf, X_train, features=features, classes=labels)
explanation = explainer.explain_global(X_test)

# Get an Explanation Client and upload the explanation
explain_client = ExplanationClient.from_run(run)
explain_client.upload_model_explanation(explanation, comment='Tabular Explanation')

print('Completed the training')
run.complete()

Overwriting train_adult_salary.py


## creating pipeline steps:

In [15]:

from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.steps import PythonScriptStep
from azureml.core import Environment
from azureml.core.runconfig import RunConfiguration
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Dataset
from azureml.data.datapath import DataPath
# Create a folder for the experiment files
experiment_folder = './'
# creating env
from azureml.core import Experiment, ScriptRunConfig, Environment
experiment_env = Environment.from_conda_specification("adult_classification_env", "D:/Adult Classification Project/Adult-Income-Classification/environment.yml")

# Register the environment 
experiment_env.register(workspace=ws)
registered_env = Environment.get(ws, 'adult_classification_env')

# get compute cluster
pipeline_cluster = ComputeTarget(workspace=ws, name='classification-cpu-cluster')

# Create a new runconfig object for the pipeline
pipeline_run_config = RunConfiguration()

# Use the compute you created above. 
pipeline_run_config.target = pipeline_cluster

# Assign the environment to the run configuration
pipeline_run_config.environment = registered_env

# Get the training dataset
adult_income_ds = ws.datasets.get("salary_classification")

# Create an OutputFileDatasetConfig (temporary Data Reference) for data passed from step 1 to step 2
prepped_data = OutputFileDatasetConfig("prepped_data")

# Step 1, Run the data prep script
analyse_step = PythonScriptStep(name = "Data Stats",
                                source_directory = experiment_folder,
                                script_name = "dataprep_experiment.py",
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)
prep_step = PythonScriptStep(name = "Prepare Data",
                                source_directory = experiment_folder,
                                script_name = "prep_adult_income.py",
                                arguments = ['--input-data', adult_income_ds.as_named_input('raw_data'),
                                             '--prepped-data', prepped_data],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

# Step 2, run the training script
train_step = PythonScriptStep(name = "Normalize,Split,Train and Register Model",
                                source_directory = experiment_folder,
                                script_name = "train_adult_salary.py",
                                arguments = ['--training-data', prepped_data.as_input()],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

print("Pipeline steps defined")

Pipeline steps defined


In [16]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

# Construct the pipeline
pipeline_steps = [analyse_step,prep_step, train_step]
pipeline = Pipeline(workspace=ws, steps=pipeline_steps)
print("Pipeline is built.")

# Create an experiment and run the pipeline
experiment = Experiment(workspace=ws, name = 'adult-income-pipeline')
pipeline_run = experiment.submit(pipeline, regenerate_outputs=True)
print("Pipeline submitted for execution.")
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion(show_output=True)

Pipeline is built.
Created step Data Stats [d4227f70][9bd84f97-b8f5-45f9-a926-69a6f3e17c6f], (This step will run and generate new outputs)Created step Prepare Data [bd4f3335][9b24ab1d-380e-4d92-9905-6182e56d6ad2], (This step will run and generate new outputs)
Created step Normalize,Split,Train and Register Model [0d1f25d9][6419f957-d111-4932-8d23-2d7cf095be65], (This step will run and generate new outputs)

Submitted PipelineRun d91d9a89-cad2-40fd-89b6-65d777dd8f9d
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/d91d9a89-cad2-40fd-89b6-65d777dd8f9d?wsid=/subscriptions/2ff60c86-4824-4a6a-bb33-beb7c5b60b23/resourcegroups/machinelearning/workspaces/akashbws&tid=191c77d0-3f43-488a-819a-6a562723ccde
Pipeline submitted for execution.


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

PipelineRunId: d91d9a89-cad2-40fd-89b6-65d777dd8f9d
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/d91d9a89-cad2-40fd-89b6-65d777dd8f9d?wsid=/subscriptions/2ff60c86-4824-4a6a-bb33-beb7c5b60b23/resourcegroups/machinelearning/workspaces/akashbws&tid=191c77d0-3f43-488a-819a-6a562723ccde
PipelineRun Status: Running


StepRunId: 12777285-daec-41ff-bd99-e01e9e38c11d
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/12777285-daec-41ff-bd99-e01e9e38c11d?wsid=/subscriptions/2ff60c86-4824-4a6a-bb33-beb7c5b60b23/resourcegroups/machinelearning/workspaces/akashbws&tid=191c77d0-3f43-488a-819a-6a562723ccde
StepRun( Prepare Data ) Status: Running

Streaming azureml-logs/20_image_build_log.txt
2022/07/23 04:04:14 Downloading source code...
2022/07/23 04:04:15 Finished downloading source code
2022/07/23 04:04:15 Creating Docker network: acb_default_network, driver: 'bridge'
2022/07/23 04:04:16 Successfully set up Docker network: acb_default_network
2022/07/23 04:04:16 Se

'Finished'

In [2]:
model = ws.models['adult_income_model']
print(model.name, 'version', model.version)

adult_income_model version 1


In [3]:
model

Model(workspace=Workspace.create(name='akashbws', subscription_id='2ff60c86-4824-4a6a-bb33-beb7c5b60b23', resource_group='machinelearning'), name=adult_income_model, id=adult_income_model:1, version=1, tags={'Training context': 'Pipeline'}, properties={'AUC': '0.9226792446565601', 'Accuracy': '0.8709182106663937', 'Precision': '0.7571976967370442'})

## Publish the pipeline

In [17]:
# Publish the pipeline from the run
published_pipeline = pipeline_run.publish_pipeline(
    name="adult-income-pipeline", description="Trains diabetes model", version="1.0")

published_pipeline

Name,Id,Status,Endpoint
adult-income-pipeline,03e3308b-2869-4ecd-81b3-e3b7c725e79a,Active,REST Endpoint


In [4]:
from azureml.pipeline.core import PublishedPipeline

published_pipeline = PublishedPipeline.get(workspace=ws, id="03e3308b-2869-4ecd-81b3-e3b7c725e79a")
print(published_pipeline.name, published_pipeline.version)

adult-income-pipeline 1.0


In [45]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

https://eastus.api.azureml.ms/pipelines/v1.0/subscriptions/2ff60c86-4824-4a6a-bb33-beb7c5b60b23/resourceGroups/machinelearning/providers/Microsoft.MachineLearningServices/workspaces/akashbws/PipelineRuns/PipelineSubmit/03e3308b-2869-4ecd-81b3-e3b7c725e79a


## Authentication Header for the REST endpoint


In [46]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()
print("Authentication header ready.")

Authentication header ready.


In [47]:
import requests

experiment_name = 'adult-income-pipeline'

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": experiment_name})
run_id = response.json()["Id"]
run_id

'2624551d-268d-4422-aadd-c44ce8087c3a'

## schedule the pipeline


In [5]:

from azureml.core import Datastore
from azureml.pipeline.core import Schedule
from azureml.core import Workspace
ws = Workspace.from_config()
training_datastore = Datastore(workspace=ws, name='workspaceblobstore')
pipeline_schedule = Schedule.create(ws, name='Reactive Training',
                                    description='trains model on data change',
                                    pipeline_id=published_pipeline.id,
                                    experiment_name='Training_Pipeline',
                                    datastore=training_datastore,
                                    path_on_datastore='dataset/salary_classification')

print("Pipeline schedule created.")

Pipeline schedule created.


In [49]:
from azureml.pipeline.core import ScheduleRecurrence, Schedule

# Submit the Pipeline every Monday at 00:00 UTC
recurrence = ScheduleRecurrence(frequency="Week", interval=1, week_days=["Monday"], time_of_day="00:00")
weekly_schedule = Schedule.create(ws, name="weekly-adult-income-training", 
                                  description="Based on time",
                                  pipeline_id=published_pipeline.id, 
                                  experiment_name='adult-income-pipeline', 
                                  recurrence=recurrence)
print('Pipeline scheduled.')

Pipeline scheduled.


In [50]:
import os

# Create a folder for the deployment files
deployment_folder = './adult_income_service'
os.makedirs(deployment_folder, exist_ok=True)
print(deployment_folder, 'folder created.')

# Set path for scoring script
script_file = 'score_adult_income.py'
script_path = os.path.join(deployment_folder,script_file)
print(script_path, 'script file created.')
print(os.getcwd())

./adult_income_service folder created.
./adult_income_service\score_adult_income.py script file created.
d:\Adult Classification Project\Adult-Income-Classification\research_env


In [51]:
%%writefile $script_path
import json
import joblib
import numpy as np
import os

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'adult_income_model.pkl')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['<=50K', '>50K']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

Overwriting ./adult_income_service\score_adult_income.py


In [52]:

from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Model

# Configure the scoring environment
service_env = Environment.get(ws, 'adult_classification_env')

model = ws.models['adult_income_model']

inference_config = InferenceConfig(source_directory=deployment_folder,
                                   entry_script=script_file,
                                   environment=service_env)

# Configure the web service container
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Deploy the model as a service
print('Deploying model...')
service_name = "adult-income-service"
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)
print(service.state)

Deploying model...
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-07-23 11:19:19+05:30 Creating Container Registry if not exists.
2022-07-23 11:19:19+05:30 Registering the environment.
2022-07-23 11:19:20+05:30 Use the existing image.
2022-07-23 11:19:21+05:30 Generating deployment configuration.
2022-07-23 11:19:22+05:30 Submitting deployment to compute.
2022-07-23 11:19:24+05:30 Checking the status of deployment adult-income-service..
2022-07-23 11:22:47+05:30 Checking the status of inference endpoint adult-income-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [53]:
from azureml.core.webservice import AciWebservice
import json

x_new = [[32,3,8536,5,4,2,7,1,2,0,0,0,15,22]]
print ('Adult Details: {}'.format(x_new[0]))
# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})
# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)
# Get the predicted class - it'll be the first (and only) one.
predicted_classes = json.loads(predictions)
print('prediction for the given adult details: ',predicted_classes[0])

Adult Details: [32, 3, 8536, 5, 4, 2, 7, 1, 2, 0, 0, 0, 15, 22]
prediction for the given adult details:  <=50K


In [54]:
help(service)

Help on AciWebservice in module azureml.core.webservice.aci object:

class AciWebservice(azureml.core.webservice.webservice.Webservice)
 |  AciWebservice(workspace, name)
 |  
 |  Represents a machine learning model deployed as a web service endpoint on Azure Container Instances.
 |  
 |  A deployed service is created from a model, script, and associated files. The resulting web service
 |  is a load-balanced, HTTP endpoint with a REST API. You can send data to this API and receive the
 |  prediction returned by the model.
 |  
 |  For more information, see `Deploy a model to Azure Container
 |  Instances <https://docs.microsoft.com/azure/machine-learning/how-to-deploy-azure-container-instance>`__.
 |  
 |  .. remarks::
 |  
 |      The recommended deployment pattern is to create a deployment configuration object with the
 |      ``deploy_configuration`` method and then use it with the ``deploy`` method of the
 |      :class:`azureml.core.model.Model` class as shown below.
 |  
 |     

In [55]:
for webservice_name in ws.webservices:
    print(webservice_name)

adult-income-service


In [56]:
from azureml.core.webservice import AciWebservice
service = azureml.core.Webservice(workspace=ws, name='adult-income-service')
print(service.scoring_uri)
print(service.swagger_uri)


http://f75c50ac-68a8-4d99-a6e1-f4a343eb355e.eastus.azurecontainer.io/score
http://f75c50ac-68a8-4d99-a6e1-f4a343eb355e.eastus.azurecontainer.io/swagger.json


In [57]:
service

AciWebservice(workspace=Workspace.create(name='akashbws', subscription_id='2ff60c86-4824-4a6a-bb33-beb7c5b60b23', resource_group='machinelearning'), name=adult-income-service, image_id=None, image_digest=None, compute_type=ACI, state=Healthy, scoring_uri=http://f75c50ac-68a8-4d99-a6e1-f4a343eb355e.eastus.azurecontainer.io/score, tags=None, properties={'azureml.git.repository_uri': 'https://github.com/AkashBorgalli/Adult-Income-Classification.git', 'mlflow.source.git.repoURL': 'https://github.com/AkashBorgalli/Adult-Income-Classification.git', 'azureml.git.branch': 'main', 'mlflow.source.git.branch': 'main', 'azureml.git.commit': '333852a2cf928690a16678f3b9d75481c757f6c9', 'mlflow.source.git.commit': '333852a2cf928690a16678f3b9d75481c757f6c9', 'azureml.git.dirty': 'True', 'hasInferenceSchema': 'False', 'hasHttps': 'False', 'authEnabled': 'False'}, created_by={'userObjectId': 'f28fb79a-1f88-47a7-8dc7-cce657d4c489', 'userPuId': '100320010799B390', 'userIdp': 'live.com', 'userAltSecId': '1

## Delete a webservice


In [ ]:
service.delete()
print ('Service deleted.')